In [ ]:
import os
import sys
import random
from langchain_community.vectorstores import Chroma
from langchain_experimental.utilities import PythonREPL
from IPython.display import display, Markdown
import jsonlines
import chromadb
import yaml

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval
from utils.agents.static_RAG_with_coding import CodeRAG

examples = []

CONFIG_PATH = os.path.join(kit_dir, "config.yaml")

In [ ]:
def load_config(filename: str) -> dict:
    try:
        with open(filename, "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(
            f"The YAML configuration file {filename} was not found."
        )
    except yaml.YAMLError as e:
        raise RuntimeError(f"Error parsing YAML file: {e}")

In [ ]:
CONFIG_PATH

In [ ]:
configs = load_config(CONFIG_PATH)

In [ ]:
configs["prompts"]["router_prompt"]

In [ ]:
documentRetrieval = DocumentRetrieval()
*_, embedding_model_info, retrieval_info, _ = documentRetrieval.get_config_info()
embeddings = documentRetrieval.load_embedding_model()

In [ ]:
vectorstore = Chroma(
    persist_directory=kit_dir + "/data/uber_lyft.chromadb",
    embedding_function=embeddings,
)

In [ ]:
# instantiate rag
rag = CodeRAG(
    configs=CONFIG_PATH,
    embeddings=embeddings,
    vectorstore=vectorstore,
    examples=examples,
)

# Initialize chains
rag.initialize()

# Build nodes
workflow = rag.create_rag_nodes()
print(workflow)

# Build graph
app = rag.build_rag_graph(workflow)

In [ ]:
# rag.display_graph(app)

In [ ]:
def log_answers(question, output_file):
    try:
        response = rag.call_rag(app, question)
    except:
        response = {}
        response["answer"] = "Failed"

    display(Markdown("---Response---"))
    display(Markdown(response["answer"]))

    if not os.path.exists(kit_dir + "/outputs"):
        os.makedirs(kit_dir + "/outputs")
    output_dir = kit_dir + "/outputs"

    with jsonlines.open(output_dir + f"/{output_file}", "a") as writer:
        writer.write({"question": question, "response": response["answer"]})

In [ ]:
questions = [
    "Could the trading price of lyft stock be volatile?",
    "What was the difference in revenue for Uber between 2020 and 2021?",
    "What was the change in revenue for Lyft for 2020 and 2021 as a percentage?",
    "Provide the business overviews for Uber and Lyft.",
    "What are the growth strategies for Lyft and Uber?",
    "How do the growth strategies differ between Lyft and Uber?",
]

In [ ]:
for question in questions:
    log_answers(question, "lyft_stock_price_Llama3_8B_Instruct.jsonl")